# Create Masterfiles for JSN prediction
Create MAsterfiles with different class definitions and exclusion criteria <br>
Split Training in OAI, MOST und CHECK


author = MV<br>
date = 2021-11-12<br>

_______________________________________

## CSV files
All Masterfiles contain data from OAI, MOST and CHECK

- Master_20JSN_XML_ex4.csv (/srv/Class_def_files/Master_20JSN_XML_ex4.csv)
    - class definition: 20% JSN/ 2 years
    - exclusion: KL 4 at baseline
    - numeric data: not normalized
    - mis sing values: included<br><br>

- Master_20JSN_XML_ex014.csv (/srv/Class_def_files/Master_20JSN_XML_ex014.csv)
    - class definition: 20% JSN/ 2 years
    - exclusion: KL 0-0, KL1-1, KL 4 at baseline
    - numeric data: not normalized and standardized
    - missing values: included<br><br>
    
- Master_10JSN_XML_ex014.csv (/srv/Master_10JSN_XML_ex014.csv)
    - class definition: 10% JSN/ 2 years
    - exclusion: KL 0-0, KL1-1, KL 4 at baseline
    - numeric data: not normalized and standardized
    - missing values: included<br><br>
    
- Nonan_Master_20JSN_XML_ex014_2.csv (/srv/Class_def_files/Nonan_Master_20JSN_XML_ex014_2.csv)
    - class definition: 20% JSN/2 years
    - exclusion: KL 0-0, KL1-1, KL 4 at baseline
    - missing values: excluded<br><br>
    
- Master_10JSN.csv (/srv/Master_10JSN.csv)
    - class definition: 10% JSN/ 2 years
    - exclusion: KL 0-0, KL1-1, KL 4 at baseline
    - numeric data: not normalized and standardized
    - missing values: excluded<br><br>

## Information in csv files

Content: Patient ID,VISIT,Laterality,ID,Study,Patient Sex, Patient Age, BMI, Knee_inj (arthritis mediaction injection, 0:no, 1:yes), Hip_symptoms (0:no, 1: yes), WOMAC_dis (disability), WOMAC_pain, WOMAC_stiff (stiffness), KL , Implant (0:no, 1: yes), osteophytes (OARSI from KOALA), KL-grade (from KOALA), sclerosis (OARSI from KOALA), other_knee_KOA (contralateral KOA, 0:no, 1: yes), class

Columns ending with "_S" are standardized 

- KL-grade_a,KL-grade_b,sclerosis_a,sclerosis_b,osteophytes_a,osteophytes_b: information of KOALA (a:year0, b:year1 or 2)

- img_path/dicom_img_path: dicom image path
- img_path_cut: ROI of image
- img_path_pro: cutted image with preprocessing (resized, normalized, Gaussian blur, Clahe)

# Imports

In [ ]:
! nvidia-smi

In [ ]:
# this defines the GPU you are using
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [ ]:
# add paths for dnn2 and labelbox-connector
import sys
sys.path.insert(1, "/srv/dnn-framework2")
sys.path.insert(1, "/srv/labelbox-connector")

In [ ]:
#general
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import logging
import cv2
import copy

#tensorflow
import tensorflow as tf
from tensorflow import keras

In [ ]:
# dnn2 framework
from framework.dnn_app.dnn_app import DnnApp
from framework.data_objects import *
from framework.dataset.dataset_collection import DatasetCollection
from framework.dataset.csv_data_helper import CsvDataHelper
from framework.batch_generator.batch_generator import BatchGenerator
from framework.models.model_generator import ModelGenerator
from framework.trainings_scheduler.trainings_scheduler import TrainingsScheduler

# dnn2 networks
from networks.models.ModelTypes import ModelTypes

In [ ]:
# labelbox-connector
from LabelBox.LB_project import LBProject
from LabelBox.LB_extractor import LBExtractor
from LabelBox.LB_parser import LBParser

from LabelBox.LB_talker import LBTalker
from LabelBox.S3_talker import S3Talker

In [ ]:
#configs
%matplotlib inline
logging.basicConfig(format='%(asc' 'time)s %(name)-25s %(level' 'name)-8s %(message)s')
logging.getLogger().setLevel(logging.INFO) # you change this to logging.DEBUG to get more logging information

# Masterfile 20% JSN and exclusion of KL4

In [ ]:
df = pd.read_csv('/srv/Class_def_files/JSN_all_XML.csv')
df1 = df.drop(columns={'Unnamed: 0','Unnamed: 0.1','Unnamed: 0.1.1','Unnamed: 0.1.1.1','coords_bbox_a','rotation_a','rotation_b','coords_bbox_b'})
df1.columns

In [ ]:
df2 = df1.dropna(subset=['class_MED','class_LAT'], how = 'all')

# exclusion of KL 4 at baseline
df3 = df2[df2['KL-grade_a'] != 4]

df3['KL-grade_a'] = df3['KL-grade_a'].replace([-1.5],np.nan)
df3['KL-grade_b'] = df3['KL-grade_b'].replace([-1.5],np.nan)

# add one general class combining class_MED and class_LAT
df3['class']=''
for i, row in df3.iterrows():
    if (df3['class_MED'][i]==1) | (df['class_LAT'][i]==1):
        df3['class'][i]=1
    else:
        df3['class'][i]=0

print('Class 0:',len(df3.loc[df3['class'] == 0]))
print('Class 1:',len(df3.loc[df3['class'] == 1]))


In [ ]:
# Add class to master csv
df_master = pd.read_csv('/srv/Master_dataset_JSN.csv')

df4 = df3.rename(columns = {'ID_a':'ID'})
df5 = df_master.merge(df4[['ID','Laterality','class_MED','class_LAT','sclerosis_a', 'KL-grade_a', 'osteophytes_a', 'sclerosis_b','KL-grade_b', 'osteophytes_b','class']], on = ['ID','Laterality'], how = 'right')
df5.to_csv('/srv/Class_def_files/temp.csv')

In [ ]:
## Eliminate bad segm (-> e)

df5 = pd.read_csv('/srv/Class_def_files/temp.csv')
a = pd.read_csv('/srv/Class_def_files/FailedKOALAxml.csv')

a = a.replace(['l','r'], ['L','R']).rename(columns={'Side': 'Laterality'})
m = a.merge(df5, on = ['Laterality', 'ID'], how = 'inner')

indices = []
for i, row in m.iterrows():
    index = m['Unnamed: 0'][i]
    indices.append(index)
    
d = df5.drop(indices, axis = 0).reset_index()
e = d.drop(columns=['index'])
    
## Eliminate implants from dataframe (438 images)(-> df_f)

imp_df = pd.read_csv('/srv/Implants.csv')

imp_df = imp_df.replace(['l','r'], ['L','R'])
m = imp_df.merge(e, on = ['Laterality', 'ID'], how = 'inner')

indices = []
for i, row in m.iterrows():
    idx = m['Unnamed: 0'][i]
    indices.append(idx)
    
df_f = e.drop(indices, axis=0)

In [ ]:
len(df_f)

### Add column for contralateral KOA 

In [ ]:
df_f.sort_values(by = 'ID')
mask = df_f.duplicated(subset=['ID'], keep = False)

# keep only bilateral images 
df_d = df_f[mask].reset_index()

df_d['other_knee_KOA']=''

# add other knee fast progressor column
for idx, row in df_d.iterrows():

    print(idx, end = '\r')
    i = idx+1 # idx = first row, i = following row 

    if df_d['ID'][i] == df_d['ID'][idx]:
        if df_d['KL-grade_a'][idx]>=3:
            df_d['other_knee_KOA'][i]= 1
        else: 
            df_d['other_knee_KOA'][i]= 0
            
        if df_d['KL-grade_a'][i]>=3:
            df_d['other_knee_KOA'][idx]= 1
        else: 
            df_d['other_knee_KOA'][idx]= 0
    else:
        continue

In [ ]:
df_new = df_f.merge(df_d[['ID','Laterality','other_knee_KOA']], on = ['ID', 'Laterality'], how = 'left')

df_new = df_new.rename(columns = {'img_path':'dicom_img_path'})

df_new = df_new.replace({'Patient Sex': {'M': 0}})
df_new = df_new.replace({'Patient Sex': {'F': 1}})
df_new = df_new.replace({'Hip_symptoms': {'No':0, 'Yes':1, '1: Yes': 1, '.: Missing Form/Incomplete Workbook':np.nan, '8':np.nan, 'P':np.nan}})
df_new = df_new.replace({'Knee_inj': {'1: Yes': 1,'.: Missing Form/Incomplete Workbook':np.nan}})

df_new = df_new.replace({'WOMAC_dis': {'M': np.nan}})
df_new = df_new.replace({'WOMAC_stiff': {'M': np.nan}})
df_new = df_new.replace({'WOMAC_pain': {'M': np.nan}})

df_new['WOMAC_dis'] = df_new['WOMAC_dis'].astype('float')
df_new['WOMAC_stiff'] = df_new['WOMAC_stiff'].astype('float')
df_new['WOMAC_pain'] = df_new['WOMAC_pain'].astype('float')
df_new['Hip_symptoms'] = df_new['Hip_symptoms'].astype('float')
df_new['Knee_inj'] = df_new['Knee_inj'].astype('float')

In [ ]:
df = df_new.copy()

# Add cut_img path to dataframe
df['img_path_cut']=''
for i, row in df.iterrows():
    print(i, end='\r')
    l = df.loc[i]['Laterality']
    fn = os.path.join('/mnt/caroline/MV/Cropped_knees', df.loc[i]['ID']+'_'+ l + '.png')
    df['img_path_cut'][i] = fn

# Add preprocessed path to dataframe 
df['img_path_pro']=''
for i, row in df.iterrows():
    print(i, end='\r')
    l = df.loc[i]['Laterality']
    fn = os.path.join('/mnt/caroline/MV/Cropped_knees/Clahe-pngs', df.loc[i]['ID']+'_'+ l + '.png')
    df['img_path_pro'][i] = fn

In [ ]:
len(df)

In [ ]:
df.count()

In [ ]:
## drop images with 'no shape'
#df = pd.read_csv('/srv/Class_def_files/Master_20JSN_XML_ex4.csv')
df = df.drop([10036, 12185, 12187, 12189, 15427, 19522, 29032, 34220, 35483, 35945])
df = df.drop(columns={'KL','Implant','Knee_inj'})

In [ ]:
df1 = df.dropna()
df1.to_csv('/srv/Class_def_files/Master_20JSN_XML_ex4_test.csv')

In [ ]:
len(df1)

In [ ]:
## drop NaN values 
df = df.dropna()
df.to_csv('/srv/Class_def_files/Master_20JSN_XML_ex4.csv')

In [ ]:
df = pd.read_csv('/srv/Class_def_files/Master_20JSN_XML_ex4_train.csv')
df.Knee_inj


## Merge with standardized data

In [ ]:
dfst = pd.read_csv('Normalized_numeric_data/StandardScaler_Master_20JSN_XML_ex4.csv')
df = pd.read_csv('/srv/Class_def_files/Master_20JSN_XML_ex4_test.csv')
dfst.columns

In [ ]:
df_new = df.merge(dfst, on=['ID','Laterality'], how='left')
df_new = df_new[['Patient ID', 'VISIT',
       'Laterality', 'ID', 'Study', 'dicom_img_path', 'Patient Sex',
       'Patient Age', 'BMI', 'Hip_symptoms', 'WOMAC_dis',
       'WOMAC_pain', 'WOMAC_stiff', 'sclerosis_a',
       'KL-grade_a', 'osteophytes_a', 'class_x', 'other_knee_KOA', 'img_path_cut',
       'img_path_pro', 'Patient Sex_S', 'Patient Age_S',
       'BMI_S', 'Hip_symptoms_S', 'WOMAC_dis_S', 'WOMAC_stiff_S',
       'WOMAC_pain_S', 'other_knee_KOA_S', 'KL-grade_a_S', 'sclerosis_a_S',
       'osteophytes_a_S']]
df_new = df_new.rename(columns={'class_x':'class'})

In [ ]:
df_new.columns

In [ ]:
df_new.to_csv('/srv/Class_def_files/Master_20JSN_XML_ex4.csv')

In [ ]:
df_new.count()

## Split into Training, Test, Tune set

In [ ]:
df = pd.read_csv('/srv/Class_def_files/Master_20JSN_XML_ex4.csv')
df.columns

In [ ]:
df = df_new.copy()
np.random.seed(137115)
train_split = int(len(df) * 0.8)
test_split = len(df) - train_split

index_test = np.sort(np.random.choice(range(0, len(df)), size=test_split, replace=False))
index_train = np.delete(range(0, len(df)), index_test)

tune_split = int(len(index_train) * 0.15)
bla = np.sort(np.random.choice(range(0, len(index_train)), size=tune_split, replace=False))
bla2 = np.delete(range(0, len(index_train)), bla)
index_tune = index_train[bla]
index_train = index_train[bla2]

dfTrain = df.iloc[index_train]
dfTune = df.iloc[index_tune]
dfTest = df.iloc[index_test]

dfTrain.to_csv('/srv/Class_def_files/Master_20JSN_XML_ex4_train_2.csv')
dfTest.to_csv('/srv/Class_def_files/Master_20JSN_XML_ex4_test_2.csv')
dfTune.to_csv('/srv/Class_def_files/Master_20JSN_XML_ex4_tune_2.csv')

In [ ]:
test = df.dropna()
len(test)

# Masterfile 20% JSN and exclusion of KL0-0, KL1-1, KL4

In [ ]:
df = pd.read_csv('/srv/Class_def_files/JSN_all_XML.csv')
df_x = df.dropna(subset=['class_MED','class_LAT'], how = 'all')
print(len(df_x))

# exclude if KL stays 0
df_y = df_x.drop(df_x[(df_x['KL-grade_a'] ==0) & (df_x['KL-grade_b'] ==0)].index)
print(len(df_y))

# exclude if KL stays 1
df_z = df_y.drop(df_y[(df_y['KL-grade_a'] ==1) & (df_y['KL-grade_b'] ==1)].index)
print(len(df_z))

# exclude if KL_a is 4 at baseline
df8 = df_z[df_z['KL-grade_a'] != 4]

df8['KL-grade_a'] = df8['KL-grade_a'].replace([-1],np.nan)
df8['KL-grade_b'] = df8['KL-grade_b'].replace([-1],np.nan)

# add one general class combining class_MED and class_LAT
df8['class']=''
for i, row in df8.iterrows():
    if (df8['class_MED'][i]==1) | (df['class_LAT'][i]==1):
        df8['class'][i]=1
    else:
        df8['class'][i]=0



In [ ]:
df9 = df8.drop(columns={'Unnamed: 0','Unnamed: 0.1','Unnamed: 0.1.1','Unnamed: 0.1.1.1'})
#df9.to_csv('/srv/Class_def_files/JSN_all_XML_2.csv')

In [ ]:
# Add class to master csv
df_master = pd.read_csv('/srv/Master_dataset_JSN.csv')

df7_tmp = df9.rename(columns = {'ID_a':'ID'})
df_c = df_master.merge(df7_tmp[['ID','Laterality','KL-grade_a','KL-grade_b','sclerosis_a','sclerosis_b','osteophytes_a','osteophytes_b','class']], on = ['ID','Laterality'], how = 'right')
df_c = df_c.drop(columns={'Implant','KL','Knee_inj'})
df_c.to_csv('/srv/Class_def_files/temp.csv')

In [ ]:
## Eliminate bad segm (-> e)

df_c = pd.read_csv('/srv/Class_def_files/temp.csv')
a = pd.read_csv('/srv/Class_def_files/FailedKOALAxml.csv')

a = a.replace(['l','r'], ['L','R']).rename(columns={'Side': 'Laterality'})
m = a.merge(df_c, on = ['Laterality', 'ID'], how = 'inner')

indices = []
for i, row in m.iterrows():
    index = m['Unnamed: 0'][i]
    indices.append(index)
    
d = df_c.drop(indices, axis = 0).reset_index()
e = d.drop(columns=['index'])
    
    
## Eliminate from dataframe (438 images)(-> df_f)

imp_df = pd.read_csv('/srv/Implants.csv')

imp_df = imp_df.replace(['l','r'], ['L','R'])
m = imp_df.merge(e, on = ['Laterality', 'ID'], how = 'inner')

indices = []
for i, row in m.iterrows():
    idx = m['Unnamed: 0'][i]
    indices.append(idx)
    
df_f = e.drop(indices, axis=0)

### Add column for contralateral KOA

In [ ]:
## Add other knee column 

df_f.sort_values(by = 'ID')
mask = df_f.duplicated(subset=['ID'], keep = False)

# keep only bilateral images 
df_d = df_f[mask].reset_index()

df_d['other_knee_KOA']=''

# add other knee fast progressor column
for idx, row in df_d.iterrows():

    print(idx, end = '\r')
    i = idx+1 # idx = first row, i = following row 

    if df_d['ID'][i] == df_d['ID'][idx]:
        if df_d['KL-grade_a'][idx]>=3:
            df_d['other_knee_KOA'][i]= 1
        else: 
            df_d['other_knee_KOA'][i]= 0
            
        if df_d['KL-grade_a'][i]>=3:
            df_d['other_knee_KOA'][idx]= 1
        else: 
            df_d['other_knee_KOA'][idx]= 0
    else:
        continue
        

In [ ]:
df_new = df_f.merge(df_d[['ID','Laterality','other_knee_KOA']], on = ['ID', 'Laterality'], how = 'left')
df_d = df_d.rename(columns = {'img_path':'dicom_img_path'})

df_new = df_new.replace({'Patient Sex': {'M': 0}})
df_new = df_new.replace({'Patient Sex': {'F': 1}})
df_new = df_new.replace({'Hip_symptoms': {'No':0, 'Yes':1, '1: Yes': 1, '.: Missing Form/Incomplete Workbook':np.nan, '8':np.nan, 'P':np.nan}})
#df_new = df_new.replace({'Knee_inj': {'1: Yes': 1,'.: Missing Form/Incomplete Workbook':np.nan}})

df_new = df_new.replace({'WOMAC_dis': {'M': np.nan}})
df_new = df_new.replace({'WOMAC_stiff': {'M': np.nan}})
df_new = df_new.replace({'WOMAC_pain': {'M': np.nan}})

df_new['WOMAC_dis'] = df_new['WOMAC_dis'].astype('float')
df_new['WOMAC_stiff'] = df_new['WOMAC_stiff'].astype('float')
df_new['WOMAC_pain'] = df_new['WOMAC_pain'].astype('float')
df_new['Hip_symptoms'] = df_new['Hip_symptoms'].astype('float')
#df_new['Knee_inj'] = df_new['Knee_inj'].astype('float')

In [ ]:
df = df_new.copy()

# Add cut_img path to dataframe
df['img_path_cut']=''
for i, row in df.iterrows():
    print(i, end='\r')
    l = df.loc[i]['Laterality']
    fn = os.path.join('/mnt/caroline/MV/Cropped_knees', df.loc[i]['ID']+'_'+ l + '.png')
    df['img_path_cut'][i] = fn

# Add preprocessed path to dataframe 
df['img_path_pro']=''
for i, row in df.iterrows():
    print(i, end='\r')
    l = df.loc[i]['Laterality']
    fn = os.path.join('/mnt/caroline/MV/Cropped_knees/Clahe-pngs', df.loc[i]['ID']+'_'+ l + '.png')
    df['img_path_pro'][i] = fn

In [ ]:
df = df.drop(columns={'Unnamed: 0', 'Unnamed: 0.1'})


In [ ]:
df.to_csv('/srv/Class_def_files/Master_20JSN_XML_ex014.csv')

## Merge with standardized data

In [ ]:
df = pd.read_csv('/srv/Class_def_files/Master_20JSN_XML_ex014.csv')
dfs = pd.read_csv('/srv/Normalized_numeric_data/StandardScaler_Master_20JSN_XML_ex014.csv')
dfs.columns

In [ ]:
df_new = df.merge(dfs, on=['ID','Laterality'], how='left')
df_new.columns

In [ ]:
df = df_new.copy()

In [ ]:
df = df.drop(columns={'Unnamed: 0_x', 'Unnamed: 0_y', 'class_y'})
df = df.rename(columns={'class_x':'class'})
df.to_csv('/srv/Class_def_files/Master_20JSN_XML_ex014.csv')

## Drop nan values

In [ ]:
df=pd.read_csv('/srv/Class_def_files/Master_20JSN_XML_ex014.csv')

df2['Patient Sex'] = df['Patient Sex'].astype('float')
df2['BMI'] = df['BMI'].astype('float')
#df2['Knee_inj'] = df['Knee_inj'].astype('float')
df2['Hip_symptoms'] = df['Hip_symptoms'].astype('float')
df2['WOMAC_dis'] = df['WOMAC_dis'].astype('float')
df2['WOMAC_pain'] = df['WOMAC_pain'].astype('float')
df2['KL-grade_a'] = df['KL-grade_a'].astype('float')

df3 = df2.dropna()
df3 = df3.reset_index()
df3.to_csv('/srv/Class_def_files/Nonan_Master_20JSN_XML_ex014_2.csv')

## Train with OAI, MOST and test with CHECK

In [ ]:
df = pd.read_csv('/srv/Class_def_files/Nonan_Master_20JSN_XML_ex014_2.csv')

In [ ]:
dfOAI = df.loc[df.Study == 'OAI']
dfMOST = df.loc[df.Study == 'MOST']
dfOAIMOST = df.loc[(df.Study == 'MOST') | (df.Study == 'OAI')]
dfCHECK = df.loc[df.Study == 'CHECK']

In [ ]:
dfCHECK.to_csv('/srv/Class_def_files/Nonan_Master_20JSN_XML_ex014_2_CHECK.csv')
dfOAIMOST.to_csv('/srv/Class_def_files/Nonan_Master_20JSN_XML_ex014_2_OAIMOST.csv')

In [ ]:
df = dfOAIMOST.copy()
np.random.seed(137115)

tune_split = int(len(df) * 0.15)
index_tune = np.sort(np.random.choice(range(0, len(df)), size=tune_split, replace=False))
index_train = np.delete(range(0, len(df)), index_tune)

dfTrain = df.iloc[index_train]
dfTune = df.iloc[index_tune]

In [ ]:
dfTrain.to_csv('/srv/Class_def_files/Nonan_Master_20JSN_XML_ex014_2_OAIMOST_train.csv')
dfTune.to_csv('/srv/Class_def_files/Nonan_Master_20JSN_XML_ex014_2_OAIMOST_tune.csv')

## Train with OAI, CHECK and test with MOST

In [ ]:
df = pd.read_csv('/srv/Class_def_files/Nonan_Master_20JSN_XML_ex014_2.csv')

In [ ]:
dfOAICHECK = df.loc[(df.Study == 'CHECK') | (df.Study == 'OAI')]


In [ ]:
df = dfOAICHECK.copy()
np.random.seed(137115)

tune_split = int(len(df) * 0.15)
index_tune = np.sort(np.random.choice(range(0, len(df)), size=tune_split, replace=False))
index_train = np.delete(range(0, len(df)), index_tune)

dfTrain = df.iloc[index_train]
dfTune = df.iloc[index_tune]

In [ ]:
dfTrain.to_csv('/srv/Class_def_files/Nonan_Master_20JSN_XML_ex014_2_OAICHECK_train.csv')
dfTune.to_csv('/srv/Class_def_files/Nonan_Master_20JSN_XML_ex014_2_OAICHECK_tune.csv')
dfMOST.to_csv('/srv/Class_def_files/Nonan_Master_20JSN_XML_ex014_2_MOST.csv')

## Train with CHECK, MOST and test with OAI

In [ ]:
df = pd.read_csv('/srv/Class_def_files/Nonan_Master_20JSN_XML_ex014_2.csv')

In [ ]:
dfMOSTCHECK = df.loc[(df.Study == 'CHECK') | (df.Study == 'MOST')]
len(dfMOSTCHECK)

In [ ]:
df = dfMOSTCHECK.copy()
np.random.seed(137115)

tune_split = int(len(df) * 0.15)
index_tune = np.sort(np.random.choice(range(0, len(df)), size=tune_split, replace=False))
index_train = np.delete(range(0, len(df)), index_tune)

dfTrain = df.iloc[index_train]
dfTune = df.iloc[index_tune]

In [ ]:
dfTrain.to_csv('/srv/Class_def_files/Nonan_Master_20JSN_XML_ex014_2_MOSTCHECK_train.csv')
dfTune.to_csv('/srv/Class_def_files/Nonan_Master_20JSN_XML_ex014_2_MOSTCHECK_tune.csv')
dfOAI.to_csv('/srv/Class_def_files/Nonan_Master_20JSN_XML_ex014_2_OAI.csv')

## Split into Training, Test, Tune set

In [ ]:
df = df3.copy()

In [ ]:
np.random.seed(137115)
train_split = int(len(df) * 0.8)
test_split = len(df) - train_split

index_test = np.sort(np.random.choice(range(0, len(df)), size=test_split, replace=False))
index_train = np.delete(range(0, len(df)), index_test)

tune_split = int(len(index_train) * 0.15)
bla = np.sort(np.random.choice(range(0, len(index_train)), size=tune_split, replace=False))
bla2 = np.delete(range(0, len(index_train)), bla)
index_tune = index_train[bla]
index_train = index_train[bla2]

dfTrain = df.iloc[index_train]
dfTune = df.iloc[index_tune]
dfTest = df.iloc[index_test]

dfTrain.to_csv('/srv/Class_def_files/Nonan_Master_20JSN_XML_ex014_2_train.csv')
dfTest.to_csv('/srv/Class_def_files/Nonan_Master_20JSN_XML_ex014_2_test.csv')
dfTune.to_csv('/srv/Class_def_files/Nonan_Master_20JSN_XML_ex014_2_tune.csv')

# Masterfile 10% JSN, exclusion of KL0-0, KL1-1, KL4

In [ ]:
df = pd.read_csv('/srv/JSN_pred_clf_pro.csv')

df = df.replace({'Patient Sex': {'M': 0}})
df = df.replace({'Patient Sex': {'F': 1}})
df = df.replace({'Hip_symptoms': {'No':0, 'Yes':1, '1: Yes': 1, '.: Missing Form/Incomplete Workbook':np.nan, '8':np.nan, 'P':np.nan}})
df = df.replace({'Knee_inj': {'1: Yes': 1,'.: Missing Form/Incomplete Workbook':np.nan}})

df = df.replace({'WOMAC_dis': {'M': np.nan}})
df = df.replace({'WOMAC_stiff': {'M': np.nan}})
df = df.replace({'WOMAC_pain': {'M': np.nan}})


In [ ]:
## adding columns of KOALA output (sclerosis, osteophytosis, KL)
add = pd.read_csv('/srv/XMLExtrKLOsteoScleroAll.csv')
df1 = df.merge(add, on = ['Laterality', 'ID'], how = 'left')


### Add column for contralateral KOA

In [ ]:
df_f = df1.copy()
## Add other knee column 

df_f.sort_values(by = 'ID')
mask = df_f.duplicated(subset=['ID'], keep = False)

# keep only bilateral images 
df_d = df_f[mask].reset_index()

df_d['other_knee_KOA']=''

# add other knee fast progressor column
for idx, row in df_d.iterrows():

    print(idx, end = '\r')
    i = idx+1 # idx = first row, i = following row 

    if df_d['ID'][i] == df_d['ID'][idx]:
        if df_d['KL-grade'][idx]>=3:
            df_d['other_knee_KOA'][i]= 1
        else: 
            df_d['other_knee_KOA'][i]= 0
            
        if df_d['KL-grade'][i]>=3:
            df_d['other_knee_KOA'][idx]= 1
        else: 
            df_d['other_knee_KOA'][idx]= 0
    else:
        continue



In [ ]:
df_new = df_f.merge(df_d[['ID','Laterality','other_knee_KOA']], on = ['ID', 'Laterality'], how = 'left')
df_new = df_new.rename(columns = {'img_path':'dicom_img_path'})

# make all nan for other_knee_KOA = 0 
df_new['other_knee_KOA'] = df_new['other_knee_KOA'].fillna(0)
df_new.count()

## Add Normalized data

In [ ]:
from sklearn import preprocessing

## StandardScaler
col_names = ['Patient Sex','Patient Age','BMI','Knee_inj','Hip_symptoms','WOMAC_dis','WOMAC_stiff','WOMAC_pain','other_knee_KOA','KL-grade','sclerosis','osteophytes']
new_col_names = ['Patient Sex_S','Patient Age_S','BMI_S','Knee_inj_S','Hip_symptoms_S','WOMAC_dis_S','WOMAC_stiff_S','WOMAC_pain_S','other_knee_KOA_S','KL-grade_S','sclerosis_S','osteophytes_S']
features = df_new[col_names]

scaler = preprocessing.StandardScaler().fit(features.values)
features = scaler.transform(features.values)
scaled_features = pd.DataFrame(features, columns = new_col_names)

scaled_features['ID'] = df_new.ID
scaled_features['Laterality'] = df_new.Laterality
scaled_features['class']= df_new['class']

## eliminate all nan values
#scaled_features = scaled_features.dropna()

In [ ]:
df2 = df_new.merge(scaled_features, on=['Laterality','ID'], how='left')


In [ ]:
df2=df2[['Patient ID', 'VISIT', 'Laterality',
       'ID', 'Study', 'dicom_img_path', 'Patient Sex', 'Patient Age', 'BMI',
       'Knee_inj', 'Hip_symptoms', 'WOMAC_dis', 'WOMAC_pain', 'WOMAC_stiff',
       'KL', 'Implant','class_x', 'osteophytes', 'KL-grade', 'sclerosis',
       'other_knee_KOA', 'Patient Sex_S', 'Patient Age_S', 'BMI_S','Knee_inj_S',
       'Hip_symptoms_S', 'WOMAC_dis_S', 'WOMAC_stiff_S', 'WOMAC_pain_S',
       'other_knee_KOA_S', 'KL-grade_S', 'sclerosis_S', 'osteophytes_S']]
df2 = df2.rename(columns={'class_x':'class'})

In [ ]:
## MinMaxScaler
col_names = ['Patient Sex','Patient Age','BMI','Knee_inj','Hip_symptoms','WOMAC_dis','WOMAC_stiff','WOMAC_pain','other_knee_KOA','KL-grade','sclerosis','osteophytes']
new_col_names = ['Patient Sex_M','Patient Age_M','BMI_M','Knee_inj_M','Hip_symptoms_M','WOMAC_dis_M','WOMAC_stiff_M','WOMAC_pain_M','other_knee_KOA_M','KL-grade_M','sclerosis_M','osteophytes_M']

features = df2[col_names]

scaler = preprocessing.MinMaxScaler().fit(features.values)
features = scaler.transform(features.values)
scaled_features = pd.DataFrame(features, columns = new_col_names)

scaled_features['ID'] = df2.ID
scaled_features['Laterality'] = df2.Laterality

## Eliminiate all nan values
#scaled_features = scaled_features.dropna()

In [ ]:
df3 = df2.merge(scaled_features, on=['Laterality','ID'], how='left')
df3.columns

In [ ]:
df3.to_csv('/srv/Master_10JSN_XML_ex014.csv')

In [ ]:
df = pd.read_csv('/srv/Master_10JSN_XML_ex014.csv')
df1 = pd.read_csv('/srv/JSN_pred_clf_pro.csv')

In [ ]:
tmp= df[['ID','Laterality','Patient Sex_S','Patient Age_S','BMI_S', 'Hip_symptoms_S', 'WOMAC_dis_S', 'WOMAC_pain_S','WOMAC_stiff_S','other_knee_KOA_S','KL-grade_S','class']]
len(tmp)

In [ ]:
tmp.dropna()

In [ ]:
df1 = df1[['Laterality','ID','img_path_pro']]
df_new=tmp.merge(df1, on = ['Laterality','ID'], how = 'left')

In [ ]:
df_new.dropna()

In [ ]:
df_new.to_csv('/srv/Master_10JSN.csv')

In [ ]:
df = df_new.copy()

In [ ]:
np.random.seed(137115)
train_split = int(len(df) * 0.8)
test_split = len(df) - train_split

index_test = np.sort(np.random.choice(range(0, len(df)), size=test_split, replace=False))
index_train = np.delete(range(0, len(df)), index_test)

tune_split = int(len(index_train) * 0.15)
bla = np.sort(np.random.choice(range(0, len(index_train)), size=tune_split, replace=False))
bla2 = np.delete(range(0, len(index_train)), bla)
index_tune = index_train[bla]
index_train = index_train[bla2]

dfTrain = df.iloc[index_train]
dfTune = df.iloc[index_tune]
dfTest = df.iloc[index_test]

dfTrain.to_csv('/srv/Master_10JSN_train.csv')
dfTest.to_csv('/srv/Master_10JSN_test.csv')
dfTune.to_csv('/srv/Master_10JSN_tune.csv')